Note:   
This notebook is meant to be open with Google Colab  
(Notebook này được thiết kế cho Google Colab)

In [ ]:
# Required installations
!pip install grpcio==1.63.0
!pip install crewai
!pip install 'crewai[tools]'
!pip install langchain-google-genai==1.0.4
!pip install langchain-anthropic==0.1.20
!pip install langchain_milvus langchain-openai langchain-community
!pip install sentence-transformers


In [1]:
from google.colab import userdata
import os

In [ ]:
# TODO: add npxdegit

## PHẦN 1: HƯỚNG DẪN VÀ SET CÁC BIẾN ĐỂ CHẠY

In [3]:
#### CÁC THẦY CÔ CÓ THỂ THAY ĐỔI CÁC BIẾN DƯỚI ĐÂY
genai = "openai"
model_name = "gpt-4o-mini"
temperature = 0.5
crew_verbose = 2

province=""
school=""
year=""

#### CÁC THẦY CÔ KHÔNG THAY ĐỔI CÁC BIẾN DƯỚI ĐÂY
os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')
os.environ["DATABSE_PUBLIC_ENDPOINT"] = userdata.get("DATABSE_PUBLIC_ENDPOINT")
os.environ["DATABASE_API_KEY"] = userdata.get("DATABASE_API_KEY")



## PHẦN 2: TẠO MỘT ĐỘT NGŨ NHÂN VIÊN (Crew)

### Đội ngũ nhân viên (Crew) 1: Dự Án Tạo Ma trận đề thi
Đội ngũ nhân viên này sẽ bao gồm 3 bạn nhân viên chính hoạt đông theo tổ chức Phân Tầng (hierarchical):
- Ngừoi Giám Sát phụ trách hướng dẫn, điều phối, và ra quyết định cho dự án
- Người Kiến Tạo phụ trách tác vụ tạo ma trận đề thi
- Người Kiểm Định phụ trách kiểm tra ma trận

In [ ]:
from agents.agents import MatrixCrew

matrix_crew = MatrixCrew()
matrix_crew.run()

In [ ]:
matrix_crew.crew

### Đội ngũ nhân viên (Crew) 2: Dự Án Tạo đề thi
Đội ngũ nhân viên này sẽ bao gồm 2 bạn nhân viên chính:
- Bạn A phụ trách tạo đề thi
- Bạn B phụ trách kiểm tra đề thi

Sau khi Đội 1 đã tạo ra được ma trận, đội 1 sẽ bàn giao kết quả đã tạo được (file `matrix.json`) cho đội 2. Đội 2 sẽ dùng kết quả này cho ra đề thi dứoi dạng html

In [3]:
from agents.agents import ExamCrew

exam_crew = ExamCrew()
exam_crew.run()

/Users/mynguyen/Repos/genai-agents-for-teachers/.venv/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(
/Users/mynguyen/Repos/genai-agents-for-teachers/.venv/lib/python3.11/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


In [4]:
exam_crew.run()

 [2024-07-28 18:58:56][DEBUG]: == Working Agent: Giáo viên tạo Bài Kiểm Tra và Bài Tập Vật Lý
 [2024-07-28 18:58:56][INFO]: == Starting Task: 1. Xác định các khái niệm và kỹ năng cần kiểm tra cho từng chủ đề động học, độ dịch chuyển
2. Tạo 2 câu hỏi trắc nghiệm để đánh giá sự hiểu biết của học sinh về từng chủ đề trong động học, độ dịch chuyển.
3. Phát triển các bài tập để học sinh thực hành và củng cố kiến thức.
4. Bao gồm các câu hỏi và bài tập ở nhiều mức độ khó khác nhau để phù hợp với trình độ của học sinh, đi từ dễ đến khó, bao gồm các bài kiểm tra 15 phút và một tiết (45 phút).
5. Đảm bảo các câu hỏi và bài tập phù hợp với tiêu chuẩn chương trình học.
6. Tất cả các output bằng tiếng Việt.


> Entering new CrewAgentExecutor chain...
Action: LessonRetrieverTool  
Action Input: {"query":"động học, độ dịch chuyển"}   

I. CHUYỂN ĐỘNG THẲNG

Chuyển động thẳng là chuyển động thường gặp trong đời sống, có quỹ đạo chuyển động là đường thẳng.

Khi vật chuyển động thẳng theo một chiều khô

CrewOutput(raw='# Đề Kiểm Tra và Bài Tập Vật Lý 9: Động Học và Độ Dịch Chuyển\n\n## Phần 1: Câu hỏi trắc nghiệm\n\n### Câu 1: Động học\n1. Vận tốc trung bình của một vật được xác định bằng công thức nào sau đây?\n   - A. \\( v = \\frac{s}{t} \\)\n   - B. \\( v = \\frac{d}{t} \\)\n   - C. \\( v = d \\cdot t \\)\n   - D. Cả A và B đều đúng\n\n**Đáp án đúng:** D\n\n### Câu 2: Độ dịch chuyển\n2. Độ dịch chuyển của một vật là gì?\n   - A. Quãng đường mà vật đã đi được\n   - B. Khoảng cách từ vị trí ban đầu đến vị trí cuối cùng\n   - C. Thời gian mà vật di chuyển\n   - D. Vận tốc của vật\n\n**Đáp án đúng:** B\n\n## Phần 2: Câu hỏi tự luận\n\n### Câu 1: Tính toán quãng đường và độ dịch chuyển\n1. Một bạn A đi từ nhà đến trường với quãng đường là 1 km hết 10 phút. Tính vận tốc trung bình của bạn A.\n\n**Hướng dẫn chấm điểm:**\n- Công thức vận tốc: \\( v = \\frac{s}{t} \\)\n- Đáp án: 6 km/h\n\n### Câu 2: Vận tốc tức thời\n2. Một xe ô tô chuyển động với vận tốc 60 km/h. Nếu xe dừng lại trong 5 g

# LỜI KẾT:
Sau khi 2 đội ngũ của chúng ta đã thực hiện xong dự án của mình, chúng ta sẽ có
- 1 bộ ma trận đề thi dứoi dạng HTML
- 1 bộ đề thi dứoi dạng HTML